### Create API Gateway for Image Retrieval Function

In [1]:
import nuclio

In [ ]:
# %run components/util/set_env.ipynb

In [2]:
%run ../util/set_env.ipynb

In [3]:
# nuclio: start-code

In [4]:
import urllib3
import requests
import json
import time
import os

# HACK for systems without production cert - shut up the no cert verification WARN
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

API_GATEWAY_NAME = f"{os.getenv('API_GATEWAY')}-{os.getenv('PROJECT')}"
API_GATEWAY_HOST = f"{API_GATEWAY_NAME}.{os.getenv('IGZ_NAMESPACE_DOMAIN')}"
API_FUNCTION = f"{os.getenv('PROJECT')}-{os.getenv('GET_IMAGE_FUNCTION')}"
DASHBOARD_URL = f"https://dashboard.{os.getenv('IGZ_NAMESPACE_DOMAIN')}"

session = requests.Session()
session.auth = (os.getenv("V3IO_USERNAME"), os.getenv("IGZ_AUTH"))
auth = session.post(f'{DASHBOARD_URL}/api/sessions', verify=False)

session_id = auth.json()['data']['id']
print(f'Session opened, your session ID is: {session_id}')

request = {
   'data':{
      'attributes':{
         'meta':{
            'name': os.getenv("API_GATEWAY"),
             "labels": {
                "entries": [
                  {
                    "value": os.getenv("V3IO_USERNAME"),
                    "key": "iguazio.com/username"
                  },
                  {
                    "value": os.getenv("PROJECT"),
                    "key": "project-name"
                  }
                ]
            },
         },
         'spec':{
            'authentication_mode':'none',
            'host': API_GATEWAY_HOST,
            'upstreams':[
               {
                  'kind':'nucliofunction',
                  'nucliofunction':{
                     'name': API_FUNCTION,
                  }
               },
            ],
            'description':f'Get image from "{os.getenv("TAGGED_VIDEO_STREAM")}"',
            'name': os.getenv("API_GATEWAY"),
         },
      },
      'type':'api_gateway'
   },
}

def handler(context, event):
    response = session.post(f'{DASHBOARD_URL}/api/api_gateways', json=request, headers={'x-igz-project-name': os.getenv("PROJECT")})
    if response.status_code == 409:
        print(f'API Gateway with name {API_GATEWAY_NAME} already exists (That\'s OK!)')
    else:
        print(json.dumps(response.json(), indent=2))

    print('Done creating API gateway!')

Session opened, your session ID is: 44293a1e-5d90-4e94-9038-5f61e8daf818


In [5]:
# nuclio: end-code

In [6]:
from mlrun import code_to_function, mount_v3io, mlconf
import os

fn = code_to_function('create-api-gateway', kind='job', handler="handler", project=os.getenv("PROJECT"))

In [7]:
fn.spec.build.image = f"docker-registry.{os.getenv('IGZ_NAMESPACE_DOMAIN')}:80/{os.getenv('DOCKER_IMAGE')}"
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1

fn.set_env('API_GATEWAY', API_GATEWAY)
fn.set_env('PROJECT', PROJECT)
fn.set_env('GET_IMAGE_FUNCTION', GET_IMAGE_FUNCTION)
fn.set_env('V3IO_USERNAME', os.getenv('V3IO_USERNAME'))
fn.set_env('IGZ_AUTH', IGZ_AUTH)
fn.set_env('IGZ_NAMESPACE_DOMAIN', os.getenv('IGZ_NAMESPACE_DOMAIN'))
fn.set_env('TAGGED_VIDEO_STREAM', TAGGED_VIDEO_STREAM)

fn.export("../yaml/create-api-gateway.yaml")

> 2020-10-28 19:54:47,102 [info] function spec saved to path: ../yaml/create-api-gateway.yaml
